SKOS Special Values

In [1]:
import pandas as pd
import numpy as np
import csv
import uuid

In [2]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/skos/SKOS_SPECIAL_VALUES.csv"

skos_sp = pd.read_csv(PATH,header=0)
display(skos_sp)

,skos:Concept,rdf:type,skos:inScheme,skos:broader,skos:closeMatch,skos:prefLabel@fr,skos:prefLabel@en,skos:altLabel@fr,skos:altLabel@en,skos:hiddenLabel,skos:definition@fr
0,ConstructionNouvelleSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Construction nouvelle,NaN,NaN,NaN,"Construction nouvelle,Nouvelle construction,CN...",Destruction d'un bâtiment qui est donc imposab...
1,DemolitionSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Démolition,NaN,NaN,NaN,"Détruite,Destruction,Démolie,Démolition",Destruction d'un bâtiment qui n'est donc plus ...
2,RuinesSV,SpecialValue,SpecialCellValuesList,NaN,NaN,En ruines,NaN,NaN,NaN,"En ruine,Ruines,En ruines","Construction enruine, n'est généralement plus ..."
3,AugmentationSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Augmentation,NaN,NaN,NaN,"Augmentation,Augtion,Augton,Augon",Modification d'un bâtiment qui entraine une au...
4,AdditionConstructionSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Addition à une construction,NaN,NaN,NaN,"addition, add C,add",Addition à une contruction existente
5,EvaluationSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Evaluation,NaN,NaN,NaN,"Evaluation,Eval",Evaluation ou réévaluation fiscale d'une parcelle
6,DiminutionSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Diminution,NaN,NaN,NaN,"Diminution,Dim,Dton",Modification d'un bâtiment qui entraine une di...
7,OmissionSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Omission,NaN,OmissionSV{{date}},NaN,"Omission,Omission depuis,OmissionSV{{date}}",Déclaration omise depuis une date donnée réint...
8,ResteSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Reste,NaN,NaN,NaN,Reste,Reste d'une parcelle modifiée (divisée ou part...
9,VoiePubliqueSV,SpecialValue,SpecialCellValuesList,NaN,NaN,Passé à la voie publique,NaN,NaN,NaN,"Voie publique,Vp",Indique d'une partie de la parcelle est passée...


In [3]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastre/specialCellValue/")
g.bind('cad_spval', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
g.bind('cad', cad)

# Iterate over each row in the DataFrame
for index, row in skos_sp.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, cad.SpecialCellValue))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:altLabel@fr']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:prefLabel@en']):
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if not pd.isnull(row['skos:altLabel@en']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if not pd.isnull(row['skos:broader']):
        g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if not pd.isnull(row['skos:definition@fr']):
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition@fr"],lang='fr')))

    if not pd.isnull(row['skos:hiddenLabel']):
        if ',' in row["skos:hiddenLabel"]:
            for hiddenLabel in row["skos:hiddenLabel"].split(','):
                g.add((subject_uri, SKOS.hiddenLabel, Literal(hiddenLabel)))
        else:
            g.add((subject_uri, SKOS.hiddenLabel, Literal(row["skos:hiddenLabel"])))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix cad_spval: <http://data.ign.fr/id/codes/cadastre/specialCellValue/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

cad_spval:AdditionConstructionSV a cad:SpecialCellValue,
        owl:NamedIndividual,
        skos:Concept ;
    skos:definition "Addition à une contruction existente"@fr ;
    skos:hiddenLabel " add C",
        "add",
        "addition" ;
    skos:inScheme cad:SpecialCellValuesList ;
    skos:prefLabel "Addition à une construction"@fr .

cad_spval:AugmentationSV a cad:SpecialCellValue,
        owl:NamedIndividual,
        skos:Concept ;
    skos:definition "Modification d'un bâtiment qui entraine une augmentation du montant de l'impôt"@fr ;
    skos:hiddenLabel "Augmentation",
        "Augon",
        "Augtion",
        "Augton" ;
    skos:inScheme cad:SpecialCellValuesList ;
    skos:prefLabel "Augmentation"@fr .

cad_spval:CelluleVide a cad:SpecialCellVal